In [1]:
import sys, getopt
import json
import pandas as pd
import os
import shutil
base_path = "D:/annotations/instances_"
sets = ['train2014', 'val2014']
names = ['sports ball', 'frisbee', 'bowl', 'orange', 'apple', 'donut']

In [2]:
for x in sets:
    file_path = f'{base_path}{x}.json'

    with open(file_path) as fp:
        data = json.load(fp)
        
    categories = pd.json_normalize(data['categories'])
    categories = categories[['name', 'id']]
    categories.name.unique()
    
    ids = categories[categories.name.isin(names)].id.values
    
    annotations = pd.json_normalize(data['annotations'])
    images_to_keep_ids = annotations[annotations['category_id'].isin(ids)]['image_id'].values

    images = pd.json_normalize(data['images'])
    
    # Get files, move them to sub-directories
    images_to_keep_file_name = images[images.id.isin(images_to_keep_ids)].file_name
    tuples = [(x,y) for x, y in zip(images_to_keep_ids, images_to_keep_file_name)]
    with open(f'D:/{x}_subset.txt', 'w') as fp:
        for x in tuples:
            fp.write(f'{x[0]},{x[1]}\n')

     new_dirpath = f'D:/{x}_subset'
    os.mkdir(new_dirpath)
    for file in images_to_keep_file_name.values:
        try:
            shutil.move(f'D:/{x}/{file}', f'{new_dirpath}/{file}')
        except:
            print(f"Couldn't find {file}")
            continue
            
    # Extract relevant annotation data for subset images
    new_data = {}
    new_data['info'] = data['info']
    new_data['images'] = [x for x in data['images'] if x['id'] in images_to_keep_ids]
    new_data['licenses'] = data['licenses']
    new_data['annotations'] = [x for x in data['annotations'] if x['id'] in images_to_keep_ids]
    new_data['categories'] = [x for x in data['categories'] if x['id'] in ids]
    
    with open(f'{end_path}/instances_{x}.json', 'w') as fp:
        json.dump(new_data, fp)
        
    
    # captions
    with open(f'D:/annotations/captions_{x}.json', 'r') as fp:
        data = json.load(fp)

    new_data = {}
    new_data['info'] = data['info']
    new_data['images'] = [x for x in data['images'] if x['id'] in images_to_keep_ids]
    new_data['licenses'] = data['licenses']
    new_data['annotations'] = [x for x in data['annotations'] if x['image_id'] in images_to_keep_ids]        
    
    with open(f'{end_path}/captions_{x}.json', 'w') as fp:
        json.dump(new_data, fp)
    
    with open(f'D:/annotations/person_keypoints_{x}.json', 'r') as fp:
        data = json.load(fp)
    new_data = {}
    new_data['info'] = data['info']
    new_data['images'] = [x for x in data['images'] if x['id'] in images_to_keep_ids]
    new_data['licenses'] = data['licenses']
    new_data['annotations'] = [x for x in data['annotations'] if x['image_id'] in images_to_keep_ids]
    new_data['categories'] = data['categories']
    

    with open(f'{end_path}/person_keypoints_{x}.json', 'w') as fp:
        json.dump(new_data, fp)